<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg" 
     width="30%" 
     align=right
     alt="Dask logo">

# Embarrassingly parallel Workloads

This notebook shows how to use Dask to parallelize embarrassingly parallel workloads where you want to apply one function to many pieces of data independently.  It will show three different ways of doing this with Dask:

1. [dask.delayed](http://dask.pydata.org/en/latest/delayed.html) 
2. [concurrent.Futures](https://dask.pydata.org/en/latest/futures.html) 
3. [dask.bag](https://dask.pydata.org/en/latest/bag.html)

This example focuses on using Dask for building large embarrassingly parallel computation as often seen in scientific communities and on High Performance Computing facilities, for example with Monte Carlo methods. This kind of simulation assume the following:

 - We have a function that runs a heavy computation given some parameters.
 - We need to compute this function on many different input parameters, each function call being independent.
 - We want to gather all the results in one place for further analysis.

## Start Dask Client for Dashboard

Starting the Dask Client will provide a dashboard which 
is useful to gain insight on the computation.  We will also need it for the
Futures API part of this example. Moreover, as this kind of computation
is often launched on super computer or in the Cloud, you will probably end
up having to start a cluster and connect a client to scale.  See 
[dask-jobqueue](https://github.com/dask/dask-jobqueue),
[dask-kubernetes](https://github.com/dask/dask-kubernetes) or 
[dask-yarn](https://github.com/dask/dask-yarn) for easy ways to achieve this
on respectively an HPC, Cloud or Big Data infrastructure.

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same time is very useful when learning.

In [17]:
from dask.distributed import Client, progress
from dask_jobqueue import SLURMCluster

# Simplify SLURMCluster parameters to basic working configuration
cluster_kwargs = {
    "cores": 1,  
    "memory": "10GB", 
    "shebang": "#!/bin/bash",
    "account": "facet",
    "walltime": "00:10:00",
    "job_script_prologue": ["source ~/.bashrc"],
    # Only basic directives, add more as needed and test each step
    "job_extra_directives": ["-q debug", "--partition=milano", "--qos=preemptable"],
}

cluster = SLURMCluster(**cluster_kwargs)
print(cluster.job_script())


slurm_jobs = 10
cluster.scale(jobs=slurm_jobs)
client = Client(cluster)


#!/bin/bash

#SBATCH -J dask-worker
#SBATCH -A facet
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=10G
#SBATCH -t 00:10:00
#SBATCH -q debug
#SBATCH --partition=milano
#SBATCH --qos=preemptable
source ~/.bashrc
/sdf/home/s/sanjeev/miniforge3/envs/dask/bin/python -m distributed.cli.dask_worker tcp://172.24.49.17:33965 --name dummy-name --nthreads 1 --memory-limit 9.31GiB --nanny --death-timeout 60



/sdf/home/s/sanjeev/miniforge3/envs/dask/lib/python3.12/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39265 instead
  warnings.warn(


## Define your computation calling function

This function does a simple operation: add all numbers of a list/array together, but it also sleeps for a random amount of time to simulate real work. In real use cases, this could call another python module, or even run an executable using subprocess module.

In [5]:
import time
import random

def costly_simulation(list_param):
    time.sleep(random.random())
    return sum(list_param)

We try it locally below

In [6]:
%time costly_simulation([1, 2, 3, 4])

CPU times: user 17.8 ms, sys: 4.93 ms, total: 22.7 ms
Wall time: 179 ms


10

## Define the set of input parameters to call the function

We will generate a set of inputs on which we want to run our simulation function. Here we use Pandas dataframe, but we could also use a simple list. Lets say that our simulation is run with four parameters called param_[a-d].

In [7]:
import pandas as pd
import numpy as np

input_params = pd.DataFrame(np.random.random(size=(500, 4)),
                            columns=['param_a', 'param_b', 'param_c', 'param_d'])
input_params.head()

,param_a,param_b,param_c,param_d
0,0.555610,0.145657,0.042182,0.010899
1,0.829836,0.114529,0.032511,0.220893
2,0.952167,0.109885,0.012851,0.520479
3,0.607278,0.621436,0.031056,0.427668
4,0.421928,0.553906,0.783059,0.971691


Without using Dask, we could call our simulation on all of these parameters using normal Python for loops.

Let's only do this on a sample of our parameters as it would be quite long otherwise.

In [8]:
results = []

In [9]:
%%time
for parameters in input_params.values[:10]:
    result = costly_simulation(parameters)
    results.append(result)

CPU times: user 352 ms, sys: 60.7 ms, total: 413 ms
Wall time: 5.53 s


In [10]:
results

[np.float64(0.7543481958336008),
 np.float64(1.1977689197898904),
 np.float64(1.5953820587430672),
 np.float64(1.6874383305208345),
 np.float64(2.7305848432877395),
 np.float64(1.0644307078976585),
 np.float64(1.901048854091663),
 np.float64(1.2634484905084877),
 np.float64(2.0256051290730124),
 np.float64(2.435369815477888)]

Note that this is not very clever as we can easily parallelize code. 

There are many ways to parallelize this function in Python with libraries like `multiprocessing`, `concurrent.futures`, `joblib` or others.  These are good first steps.  Dask is a good second step, especially when you want to scale across many machines.


## Use [Dask Delayed](http://dask.pydata.org/en/latest/delayed.html) to make our function lazy

We can call `dask.delayed` on our funtion to make it lazy.  Rather than compute its result immediately, it records what we want to compute as a task into a graph that we'll run later on parallel hardware. Using `dask.delayed` is a relatively straightforward way to parallelize an existing code base, even if the computation isn't embarrassingly parallel like this one. 

Calling these lazy functions is now almost free.  In the cell below we only construct a simple graph.

In [11]:
import dask
lazy_results = []

In [12]:
%%time

for parameters in input_params.values[:10]:
    lazy_result = dask.delayed(costly_simulation)(parameters)
    lazy_results.append(lazy_result)

CPU times: user 2.65 ms, sys: 854 μs, total: 3.5 ms
Wall time: 2.63 ms


In [13]:
lazy_results[0]

Delayed('costly_simulation-2006b1f2-9487-496f-aa45-64eb6d0b3f09')

## Run in parallel

The `lazy_results` list contains information about ten calls to `costly_simulation` that have not yet been run.  Call `.compute()` when you want your result as normal Python objects.

If you started `Client()` above then you may want to watch the status page during computation.

In [14]:
%time dask.compute(*lazy_results)

CPU times: user 431 ms, sys: 71.3 ms, total: 502 ms
Wall time: 4.56 s


(np.float64(0.7543481958336008),
 np.float64(1.1977689197898904),
 np.float64(1.5953820587430672),
 np.float64(1.6874383305208345),
 np.float64(2.7305848432877395),
 np.float64(1.0644307078976585),
 np.float64(1.901048854091663),
 np.float64(1.2634484905084877),
 np.float64(2.0256051290730124),
 np.float64(2.435369815477888))

Notice that this was faster than running these same computations sequentially with a for loop.  

We can now run this on all of our input parameters:

In [15]:
import dask
lazy_results = []

for parameters in input_params.values:
    lazy_result = dask.delayed(costly_simulation)(parameters)
    lazy_results.append(lazy_result)
    
futures = dask.persist(*lazy_results)  # trigger computation in the background

In [16]:
client

<Client: 'tcp://172.24.49.17:39695' processes=0 threads=0, memory=0 B>

To make this go faster, we can add additional workers.

(although we're still only working on our local machine, this is more practical when using an actual cluster)

In [ ]:
#client.cluster.scale(100)  # ask for a hundred 4-thread workers

By looking at the Dask dashboard we can see that Dask spreads this work around our cluster, managing load balancing, dependencies, etc..

Then get the result:

In [ ]:
%time
results = dask.compute(*futures)
results[:5]

## Using the [Futures API](http://dask.pydata.org/en/latest/futures.html)

The same example can be implemented using Dask's Futures API by using the `client` object itself.  For our use case of applying a function across many inputs both Dask delayed and Dask Futures are equally useful.  The Futures API is a little bit different because it starts work immediately rather than being completely lazy.

For example, notice that work starts immediately in the cell below as we submit work to the cluster:

In [ ]:
futures = []
for parameters in input_params.values:
    future = client.submit(costly_simulation, parameters)
    futures.append(future)

We can explicitly wait until this work is done and gather the results to our local process by calling `client.gather`:

In [ ]:
%time
results = client.gather(futures)
results[:5]

But the code above can be run in fewer lines with `client.map()` function, allowing to call a given function on a list of parameters.

As for delayed, we can only start the computation and not wait for results by not calling `client.gather()` right now.

It shall be noted that as Dask cluster has already performed tasks launching `costly_simulation` with Futures API on the given input parameters, the call to `client.map()` won't actually trigger any computation, and just retrieve already computed results.

In [ ]:
futures = client.map(costly_simulation, input_params.values)

Then just get the results later:

In [ ]:
results = client.gather(futures)
len(results)

In [ ]:
print(results[0])

We encourage you to watch the [dashboard's status page](http://127.0.0.1:8787) to watch on going computation.

## Doing some analysis on the results

One of the interests of Dask here, outside from API simplicity, is that you are able to gather the result for all your simulations in one call.  There is no need to implement a complex mechanism or to write individual results in a shared file system or object store.

Just get your result, and do some computation.

Here, we will just get the results and expand our initial dataframe to have a nice view of parameters vs results for our computation

In [ ]:
output = input_params.copy()
output['result'] = pd.Series(results, index=output.index)
output.sample(5)

Then we can do some nice statistical plots or save result locally with pandas interface here

In [ ]:
%matplotlib inline
output['result'].plot()

In [ ]:
output['result'].mean()

In [ ]:
filtered_output = output[output['result'] > 2]
print(len(filtered_output))
filtered_output.to_csv('/tmp/simulation_result.csv')

## Handling very large simulation with [Bags](http://dask.pydata.org/en/latest/bag.html)

The methods above work well for a size of input parameters up to about 100,000.  Above that, the Dask scheduler has trouble handling the amount of tasks to schedule to workers.  The solution to this problem is to bundle many parameters into a single task.
You could do this either by making a new function that operated on a batch of parameters and using the delayed or futures APIs on that function.  You could also use the Dask Bag API.  This is described more in the documentation about [avoiding too many tasks](http://dask.pydata.org/en/latest/delayed-best-practices.html#avoid-too-many-tasks).

Dask Bags hold onto large sequences in a few partitions.  We can convert our `input_params` sequence into a `dask.bag` collection, asking for fewer partitions (so at most 100,000, which is already huge), and apply our function on every item of the bag.

In [ ]:
import dask.bag as db
b = db.from_sequence(list(input_params.values), npartitions=100)
b = b.map(costly_simulation)

In [ ]:
%time results_bag = b.compute()

Looking on Dashboard here, you should see only 100 tasks to run instead of 500, each taking 5x more time in average, because each one is actually calling our function 5 times.

In [ ]:
np.all(results) == np.all(results_bag)

In [ ]:
client

In [ ]:

client.shutdown()

In [ ]:
client